In [1]:
import pandas as pd
import pickle

## Scrape and clean the data from Wikipedia

Instead of using BeautifulSoup, `pandas` has a convenient built-in function for scraping HTML tables: `pd.read_html()`. We'll use that, declare the first row of each table as its header, and select the first table:

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)[0]
print(df.shape)
df.head()

(288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Then do some quick cleaning:

In [3]:
# Excluding postcodes where the borough isn't assigned
df = df[df['Borough'] != "Not assigned"]

# Americanizing spelling for my own convenience
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

# Naming the Neighborhood after the borough where the latter is assigned but the former isn't
mask = df['Neighborhood'] == 'Not assigned'
df.loc[mask, 'Neighborhood'] = df.loc[mask, 'Borough']

# Cleaning up after the dropped rows
df.reset_index(drop=True, inplace=True)

print(df.shape)
df.head()

(211, 3)


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Consolidate postal code info

`DataFrame.groupby().agg()` allows us to group together like Postcodes and map different functions to the associated values in each column.

For the the Borough column, we  assuming that the first value we encounter is the only unqiue value for each postcode, but just to be safe we'll test it:

In [4]:
(df.groupby('Postcode').nunique() == 1).all()

Postcode         True
Borough          True
Neighborhood    False
dtype: bool

Yes, each Postcode maps 1-to-1 to a Borough. That's good; it means we can use the `GroupBy.first()` method to resolve Postcode and Borough.

For the Neighborhood column, we'll define a quick `lambda` function to join all of the Neighborhoods and separate them wil columns: `lambda x: ', '.join(x)`, and apply that.

We group by Postcode and fill out the `agg` method, renaming the Neighborhood column to Neighborhoods:

In [5]:
postcodes = (df.groupby('Postcode', as_index=False)
             .agg({'Borough':'first','Neighborhood':lambda x: ', '.join(x)})
             .rename(columns={'Neighborhood':'Neighborhoods'}))

print(postcodes.shape)
postcodes.head()

(103, 3)


,Postcode,Borough,Neighborhoods
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
pickle.dump(postcodes, open('postcodes.dataframe.p', 'wb'))

In [7]:
print(f"There are {postcodes.shape[0]} rows in the postcode DataFrame.")

There are 103 rows in the postcode DataFrame.
